In [1]:
#Importing and Loading the train.csv file
import pandas as pd
data = pd.read_csv("data/train.csv", index_col='Index')
data.head()

,LossDescription,ResultingInjuryDesc,PartInjuredDesc,Cause - Hierarchy 1,Body Part - Hierarchy 1
Index,,,,,
577,EE while helping the children clean up after l...,Fall Or Slip Injury,Lower Extremities,"Fall, Slip or Trip Injury",Lower Extremities
1867,Clmt was putting bread trays on bottom of brea...,NaN,NaN,Burn or Scald - Heat or Cold Exposures - Conta...,Neck
3530,He got off of he forklift and did not secure p...,"Motor Vehicle, NOC","Foot-Metatarsals, Heel excl Ankle or Toe",Motor Vehicle,Lower Extremities
583,slammed left finger in closet,Struck Or Injured By,Upper Extremities,Struck or Injured by,Upper Extremities
1711,the employee was digging a tre; strain; lower ...,NaN,NaN,Strain or Injury by,Trunk


In [2]:
# Function to merge the columns LossDescription, ResultingInjuryDesc, PartInjuredDesc
# into a single column called description
def merge_cols(row):
    txt = ""
    for i in row:
        if isinstance(i, str):
            txt = txt+', '+i
    return txt[2:]

data['description'] = data.iloc[:, :4].apply(lambda x : merge_cols(x), axis=1)
data.head()

,LossDescription,ResultingInjuryDesc,PartInjuredDesc,Cause - Hierarchy 1,Body Part - Hierarchy 1,description
Index,,,,,,
577,EE while helping the children clean up after l...,Fall Or Slip Injury,Lower Extremities,"Fall, Slip or Trip Injury",Lower Extremities,EE while helping the children clean up after l...
1867,Clmt was putting bread trays on bottom of brea...,NaN,NaN,Burn or Scald - Heat or Cold Exposures - Conta...,Neck,Clmt was putting bread trays on bottom of brea...
3530,He got off of he forklift and did not secure p...,"Motor Vehicle, NOC","Foot-Metatarsals, Heel excl Ankle or Toe",Motor Vehicle,Lower Extremities,He got off of he forklift and did not secure p...
583,slammed left finger in closet,Struck Or Injured By,Upper Extremities,Struck or Injured by,Upper Extremities,"slammed left finger in closet, Struck Or Injur..."
1711,the employee was digging a tre; strain; lower ...,NaN,NaN,Strain or Injury by,Trunk,the employee was digging a tre; strain; lower ...


In [3]:
#Removing the First three columns as we have already merged them into one single column
data2 = data.drop(columns=['LossDescription','ResultingInjuryDesc','PartInjuredDesc'])

In [4]:
data2.head()

,Cause - Hierarchy 1,Body Part - Hierarchy 1,description
Index,,,
577,"Fall, Slip or Trip Injury",Lower Extremities,EE while helping the children clean up after l...
1867,Burn or Scald - Heat or Cold Exposures - Conta...,Neck,Clmt was putting bread trays on bottom of brea...
3530,Motor Vehicle,Lower Extremities,He got off of he forklift and did not secure p...
583,Struck or Injured by,Upper Extremities,"slammed left finger in closet, Struck Or Injur..."
1711,Strain or Injury by,Trunk,the employee was digging a tre; strain; lower ...


In [5]:
#Checking for Null Values
data2.isna().sum()

Cause - Hierarchy 1         26
Body Part - Hierarchy 1    259
description                  0
dtype: int64

In [6]:
#Converting all the test values to Lower-case
for columns in data2.columns:
    data2.loc[:, columns] = data2.loc[:, columns].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [7]:
#Splitting the dataframe into two and removing null values, for separate classification
train_cause = data2.drop(columns=['Body Part - Hierarchy 1'])
train_bodypart = data2.drop(columns=['Cause - Hierarchy 1'])
train_cause = train_cause.dropna()
train_bodypart = train_bodypart.dropna()

In [8]:
train_bodypart.head()

,Body Part - Hierarchy 1,description
Index,,
577,lower extremities,ee while helping the children clean up after l...
1867,neck,clmt was putting bread trays on bottom of brea...
3530,lower extremities,he got off of he forklift and did not secure p...
583,upper extremities,"slammed left finger in closet, struck or injur..."
1711,trunk,the employee was digging a tre; strain; lower ...


In [9]:
train_cause.head()

,Cause - Hierarchy 1,description
Index,,
577,"fall, slip or trip injury",ee while helping the children clean up after l...
1867,burn or scald - heat or cold exposures - conta...,clmt was putting bread trays on bottom of brea...
3530,motor vehicle,he got off of he forklift and did not secure p...
583,struck or injured by,"slammed left finger in closet, struck or injur..."
1711,strain or injury by,the employee was digging a tre; strain; lower ...


In [10]:
#collection the unique values to convert them into labels
causes = list(train_cause.iloc[:, 0].unique())
causes.sort()
bodyparts = list(train_bodypart.iloc[:, 0].unique())
bodyparts.sort()
print(len(causes), causes)
print(len(bodyparts), bodyparts)

12 ['burn or scald - heat or cold exposures - contact with', 'caught in, under or between', 'cut, puncture, scrape injured by', 'fall, slip or trip injury', 'fall, slip, or trip injury', 'includes freezing', 'misc', 'motor vehicle', 'rubbed or abraded by', 'strain or injury by', 'striking against or stepping on', 'struck or injured by']
7 ['head', 'lower extremities', 'misc', 'multiple body parts', 'neck', 'trunk', 'upper extremities']


In [11]:
#Creating a new column called label
train_cause['label'] = train_cause.iloc[:, 0].apply(lambda x: causes.index(x))
train_bodypart['label'] = train_bodypart.iloc[:, 0].apply(lambda x: bodyparts.index(x))

In [12]:
train_cause.head()

,Cause - Hierarchy 1,description,label
Index,,,
577,"fall, slip or trip injury",ee while helping the children clean up after l...,3
1867,burn or scald - heat or cold exposures - conta...,clmt was putting bread trays on bottom of brea...,0
3530,motor vehicle,he got off of he forklift and did not secure p...,7
583,struck or injured by,"slammed left finger in closet, struck or injur...",11
1711,strain or injury by,the employee was digging a tre; strain; lower ...,9


In [13]:
# Saving only the description and label columns into a csv files
train_cause.iloc[:, 1:].to_csv("./data/train_cause.csv")
train_bodypart.iloc[:, 1:].to_csv("./data/train_bodypart.csv")